In [12]:
import os
os.environ['PINECONE_API_KEY'] = "xxx"
openai_api_key = os.getenv('OPENAI_API_KEY')
# Set the API key
os.environ['OPENAI_API_KEY'] = "xxx"

# Now you can use the API key
openai_api_key = os.getenv('OPENAI_API_KEY')

In [20]:
import warnings
warnings.filterwarnings('ignore', module='langchain')

### Basic Prompt

In [8]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import(
    AIMessage,
    HumanMessage,
    SystemMessage
)


In [14]:
text= """
Avion Rewards, the largest proprietary loyalty program
in Canada, was recognized as the International Loyalty
Program of the Year in Americas and Best Loyalty /
Benefits in a Financial Product at the 2023 International
Loyalty Awards; In 2023, Avion Rewards expanded
access to all Canadians regardless of where they bank
or shop, bringing unparalleled savings and flexibility
and making a fundamental shift in how the rewards
program delivers benefits
Announced a new loyalty partnership with METRO Inc.
with the launch of the no annual fee moi RBC® Visa‡
credit card, which earns Moi‡ points on all purchases as
well as allows cardholders access to unique savings
and offers from Avion Rewards and its extensive
merchant partner network
Launched Canada’s new First Home Savings Account
(FHSA) in April 2023 to help Canadians save tax free for
their first home, making it available as quickly and
conveniently as possible through multiple channels,
including RBC Direct Investing, RBC InvestEase® and
RBC in-branch advisors
Acquired OJOHome Canada Ltd., which operates a
comprehensive real estate technology platform,
HousefulTM (formerly OJO), bolstered by artificial
intelligence (AI), to further streamline the home buying
journey for Canadians while supporting them at every
stage with intuitive, digitally-enabled and insightsdriven
experiences
Continued to build world-class capabilities through the
RBC PayEdgeTM platform, leveraging data to increase
strategic value through industry and client-specific
insights to offer superior working capital solutions to
our business clients
Maintained our focus on key high-growth and highvalue
segments, such as youth and young adults,
newcomers, business owners, healthcare professionals,
retirees, and HNW clients
Drove significant new-to-RBC client acquisition from
newcomers and new partnerships, including ICICI Bank
Canada
"""

messages = [
    SystemMessage(content='You are an expert copywriter with expertize in summarizing documents'),
    HumanMessage(content=f'Please provide a short and concise summary of the following text:\n TEXT: {text}')
]

llm = ChatOpenAI(temperature=0, model_name='gpt-3.5-turbo', openai_api_key=openai_api_key)



In [16]:
llm.get_num_tokens(text)

404

In [22]:
summary_output = llm(messages)
print(summary_output.content)

Avion Rewards, Canada's largest loyalty program, won International Loyalty Program of the Year and Best Loyalty/Benefits in a Financial Product at the 2023 International Loyalty Awards. They expanded access to all Canadians, launched a new no annual fee credit card with METRO Inc., introduced Canada's First Home Savings Account, acquired OJOHome Canada Ltd., enhanced capabilities with RBC PayEdgeTM, and focused on key high-growth segments. They also drove new client acquisition from newcomers and new partnerships.


### Summarizing Using Prompt Templates

In [25]:
from langchain import PromptTemplate
from langchain.chains import LLMChain

In [31]:
template = '''
Write a concise and short summary of the following text:
TEXT: `{text}`
Translate the summary to {language}.
'''
prompt = PromptTemplate(
    input_variables=['text', 'language'],
    template=template
)

In [33]:
llm.get_num_tokens(prompt.format(text=text, language='English'))

425

In [35]:
chain = LLMChain(llm=llm, prompt=prompt)
summary = chain.run({'text': text, 'language':'Chinese'})

In [37]:
print(summary)

加拿大最大的专有忠诚计划Avion Rewards被认可为2023年国际忠诚奖美洲区年度国际忠诚计划和最佳金融产品忠诚/福利奖。他们扩大了对所有加拿大人的访问，无论他们在哪里银行或购物，为他们带来了无与伦比的节省和灵活性，并在奖励计划提供福利方面进行了根本性转变。他们与METRO Inc.宣布了新的忠诚合作伙伴关系，推出了无年费的moi RBC® Visa‡信用卡，该卡可在所有购买上赚取Moi‡积分，并允许持卡人从Avion Rewards及其广泛的商户合作伙伴网络中获得独特的节省和优惠。他们还推出了加拿大的新首次购房储蓄账户（FHSA），以帮助加拿大人为他们的第一套房子免税储蓄，并通过多种渠道（包括RBC直接投资、RBC InvestEase®和RBC分行顾问）尽快方便地提供。他们收购了OJOHome Canada Ltd.，该公司运营着一个综合的房地产技术平台HousefulTM（前身为OJO），通过人工智能（AI）进一步简化了加拿大人的购房之旅，同时通过直观、数字化和见解驱动的体验在每个阶段支持他们。他们通过RBC PayEdgeTM平台继续建设世界一流的能力，利用数据增加战略价值，通过行业和客户特定的见解为我们的商业客户提供优越的营运资本解决方案。他们继续专注于关键的高增长和高价值领域，如青少年和年轻成年人、新移民、企业主、医疗保健专业人士、退休人员和高净值客户。他们通过新移民和新合作伙伴（包括ICICI Bank Canada）实现了大量新客户的获取。


### Summarizing using SuffDocumentChain

In [42]:
from langchain import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain.chains.summarize import load_summarize_chain
from langchain.docstore.document import Document


In [46]:
with open('sj.txt', encoding='utf-8') as f:
    text = f.read()

# text

docs = [Document(page_content=text)]
llm = ChatOpenAI(temperature=0, model_name='gpt-3.5-turbo')

In [48]:
template = '''Write a concise and short summary of the following text.
TEXT: `{text}`
'''
prompt = PromptTemplate(
    input_variables=['text'],
    template=template
)

In [50]:
chain = load_summarize_chain(
    llm,
    chain_type='stuff',
    prompt=prompt,
    verbose=False
)
output_summary = chain.run(docs)

In [52]:
print(output_summary)

The speaker shares three stories from his life during a commencement speech. The first story is about connecting the dots and dropping out of college, leading to unexpected opportunities. The second story is about love and loss, including being fired from the company he co-founded. The third story is about facing death and the importance of following one's heart and intuition. The speaker encourages the audience to stay hungry and stay foolish as they begin anew.


### Summarizing Large Documents Using map_reduce

In [57]:
from langchain import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain.chains.summarize import load_summarize_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [59]:
with open('sj.txt', encoding='utf-8') as f:
    text = f.read()

llm = ChatOpenAI(temperature=0, model_name='gpt-3.5-turbo')

In [61]:
llm.get_num_tokens(text)

2653

In [69]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=50)
chunks = text_splitter.create_documents([text])

In [71]:
len(chunks)

2

In [73]:
# chain_type is map reduce
chain = load_summarize_chain(
    llm,
    chain_type='map_reduce',
    verbose=False
)
output_summary = chain.invoke(chunks)

In [74]:
# output_summary is a dict with 2 keys: 'input_documents' and 'output_text'
# displaying the summary
print(output_summary['output_text'])

Steve Jobs shares three stories from his life in his commencement speech, highlighting the importance of following one's passion, not settling, and living each day as if it were your last. He reflects on the inevitability of death and encourages the audience to stay hungry and foolish, embrace new opportunities, and follow their intuition. Jobs also references The Whole Earth Catalog's message of staying curious and open-minded.


In [77]:
chain.llm_chain.prompt.template

'Write a concise summary of the following:\n\n\n"{text}"\n\n\nCONCISE SUMMARY:'

In [79]:
chain.combine_document_chain.llm_chain.prompt.template

'Write a concise summary of the following:\n\n\n"{text}"\n\n\nCONCISE SUMMARY:'

### Map_reduce with Custom Prompts

In [151]:
map_prompt = '''
Write a short and concise summary of the following:
Text: `{text}`
CONCISE SUMMARY:
'''
map_prompt_template = PromptTemplate(
    input_variables=['text'],
    template=map_prompt
)

In [153]:
combine_prompt = '''
Write a concise summary of the following text that covers the key points.
Add a title to the summary.
Start your summary with an INTRODUCTION PARAGRAPH that gives an overview of the topic FOLLOWED
by BULLET POINTS if possible AND end the summary with a CONCLUSION PHRASE.
Text: `{text}`
'''
combine_prompt_template = PromptTemplate(template=combine_prompt, input_variables=['text'])

In [155]:
summary_chain = load_summarize_chain(
    llm=llm,
    chain_type='map_reduce',
    map_prompt=map_prompt_template,
    combine_prompt=combine_prompt_template,
    verbose=False
)
output = summary_chain.invoke(chunks)

In [156]:
# output is a dict with 2 keys: 'input_documents' and 'output_text'
# displaying the summary
print(output['output_text'])

Title: Exploring the Diverse Applications of Generative AI

Introduction:
Generative AI has revolutionized various industries by creating novel content in text, images, video, and more. This paper surveys over 350 generative AI applications across different domains to showcase the capabilities and impact of models like ChatGPT and DALL-E.

Key Points:
- Generative AI models can transform input formats into multimedia outputs like text, audio, and video, benefiting industries such as advertising, entertainment, and education.
- The review of generative AI technologies highlights their uses in text, images, video, 3D, and more, offering valuable insights for researchers and professionals.
- The evolving landscape of generative AI provides endless possibilities for content creation and transformation, shaping industries and providing innovative solutions for diverse needs.
- Generative AI applications span across sectors like art, business, biotechnology, and design, categorized into text

### Summarizing Using the refine Chain

In [95]:
from langchain_openai import ChatOpenAI
from langchain import PromptTemplate
from langchain.chains.summarize import load_summarize_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter
# from langchain.document_loaders import UnstructuredPDFLoader
from langchain.document_loaders import PyPDFLoader

In [147]:
loader = PyPDFLoader('A_survey_of_Generative_AI_Applications.pdf')
data = loader.load()

In [148]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=100)
chunks = text_splitter.split_documents(data)

In [129]:
len(chunks)

3

In [131]:
llm = ChatOpenAI(temperature=0, model_name='gpt-3.5-turbo')

In [135]:
chain = load_summarize_chain(
    llm=llm,
    chain_type='refine',
    verbose=False
)
output_summary = chain.invoke(chunks)

In [137]:
print(output_summary)

{'input_documents': [Document(metadata={'source': 'A_survey_of_Generative_AI_Applications.pdf', 'page': 0}, page_content='arXiv:2306.02781v1  [cs.LG]  5 Jun 2023A survey of Generative AI Applications\nRoberto Gozalo-Brizuela, Eduardo C. Garrido-Merch´ an\nQuantitative Methods Department, Universidad Pontiﬁcia C omillas, Madrid, Spain\n201905616@alu.comillas.edu, ecgarrido@icade.comillas .edu\nAbstract. Generative AI has experienced remarkable growth in recent\nyears, leading to a wide array of applications across divers e domains. In\nthis paper, we present a comprehensive survey of more than 35 0 gen-\nerative AI applications, providing a structured taxonomy a nd concise\ndescriptions of various unimodal and even multimodal gener ative AIs.\nThe survey is organized into sections, covering a wide range of unimodal\ngenerative AI applications such as text, images, video, gam ing and brain\ninformation. Our survey aims to serve as a valuable resource for re-\nsearchers and practitioners 

### Refine With Custom Prompts

In [141]:
prompt_template = """Write a concise summary of the following extracting the key information:
Text: `{text}`
CONCISE SUMMARY:"""
initial_prompt = PromptTemplate(template=prompt_template, input_variables=['text'])

refine_template = '''
    Your job is to produce a final summary.
    I have provided an existing summary up to a certain point: {existing_answer}.
    Please refine the existing summary with some more context below.
    ------------
    {text}
    ------------
    Start the final summary with an INTRODUCTION PARAGRAPH that gives an overview of the topic FOLLOWED
    by BULLET POINTS if possible AND end the summary with a CONCLUSION PHRASE.
    
'''
refine_prompt = PromptTemplate(
    template=refine_template,
    input_variables=['existing_answer', 'text']
)


In [143]:
chain = load_summarize_chain(
    llm=llm,
    chain_type='refine',
    question_prompt=initial_prompt,
    refine_prompt=refine_prompt,
    return_intermediate_steps=False
    
)
output_summary = chain.invoke(chunks)

In [144]:
# displaying the refined summary
print(output_summary['output_text'])

Introduction:
Generative AI has rapidly advanced in recent years, showcasing its transformative potential across various industries such as art, business, biotechnology, and design. This paper provides a comprehensive survey of over 350 generative AI applications, categorizing them based on the output produced, the context of use, and the business applications.

Key Points:
- The survey covers 13 categories of generative AI applications, including text, images, video, 3D models, and code and software.
- Generative AI models in the text category focus on creating and manipulating natural language text, including chatbots, text-writing assistants, and scientific language models.
- In the images category, generative AI technologies concentrate on generating visual images, such as artistic image creation, realistic image creation, and image editing.
- Video category models aim to create and manipulate video content, including video creation, post-production, text-to-scene generation, and i

### Summarizing Using LangChain Agents

In [160]:
from langchain_openai import ChatOpenAI
from langchain.agents import initialize_agent, Tool
from langchain.utilities import WikipediaAPIWrapper

In [167]:
llm = ChatOpenAI(temperature=0, model_name='gpt-3.5-turbo')
wikipedia = WikipediaAPIWrapper()

In [169]:
tools = [
    Tool(
        name="Wikipedia", 
        func=wikipedia.run,
        description="Useful for when you need to get information from wikipedia about a single topic"
    )
]

In [171]:
agent_executor = initialize_agent(tools, llm, agent='zero-shot-react-description', verbose=True)

In [175]:
output = agent_executor.invoke('Can you please provide a short summary of Georgia Tech?')



> Entering new AgentExecutor chain...
I should use Wikipedia to get a brief summary of Georgia Tech.
Action: Wikipedia
Action Input: Georgia Institute of Technology
Observation: Page: Georgia Tech
Summary: The Georgia Institute of Technology (commonly referred to as Georgia Tech and GT or, in the state of Georgia, as Tech or the Institute) is a public research university and institute of technology in Atlanta, Georgia. Established in 1885, it is part of the University System of Georgia and has satellite campuses in Savannah, Georgia; Metz, France; Shenzhen, China; and Singapore.
The school was founded as the Georgia School of Technology as part of Reconstruction efforts to build an industrial economy in the Southern United States after the Civil War. Initially, it offered only a degree in mechanical engineering. By 1901, its curriculum had expanded to include electrical, civil, and chemical engineering. In 1948, the school changed its name to reflect its evolution from a trade school

In [177]:
print(output['output'])

The Georgia Institute of Technology (Georgia Tech) is a public research university and institute of technology in Atlanta, Georgia. It was established in 1885 and is part of the University System of Georgia.
